# Chapter 6: Decision Trees 

In [1]:
import numpy as np

## 6.1 Training and Visualizing a Decision Tree

The following code trains a `DecisionTreeClassifier` on the iris dataset.

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [4]:
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

You can visualize the trained Decision Tree by first using `export_graphviz()` to output a graph definition file called *'iris_tree.dot'*.

> Note: Visualizing the graph needs Graphviz package's "dot" command. **See book for graph picture.**

In [6]:
from sklearn.tree import export_graphviz

In [10]:
export_graphviz(
    tree_clf,
    out_file=None,
    # out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

'digraph Tree {\nnode [shape=box, style="filled, rounded", color="black", fontname=helvetica] ;\nedge [fontname=helvetica] ;\n0 [label="petal width (cm) <= 0.8\\ngini = 0.667\\nsamples = 150\\nvalue = [50, 50, 50]\\nclass = setosa", fillcolor="#ffffff"] ;\n1 [label="gini = 0.0\\nsamples = 50\\nvalue = [50, 0, 0]\\nclass = setosa", fillcolor="#e58139"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="petal width (cm) <= 1.75\\ngini = 0.5\\nsamples = 100\\nvalue = [0, 50, 50]\\nclass = versicolor", fillcolor="#ffffff"] ;\n0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;\n3 [label="gini = 0.168\\nsamples = 54\\nvalue = [0, 49, 5]\\nclass = versicolor", fillcolor="#4de88e"] ;\n2 -> 3 ;\n4 [label="gini = 0.043\\nsamples = 46\\nvalue = [0, 1, 45]\\nclass = virginica", fillcolor="#843de6"] ;\n2 -> 4 ;\n}'

## 6.2 Making Predictions

To classify a new flower,

1. Start at the **root node** (depth 0).
2. Is petal length smaller than 2.45cm?
3. **Yes**, move down to root's left child node (depth 1, left).
4. **No**, move down to the root's right child node (depth 1, right).
5. Is child node a **leaf node** (does not have any more child nodes)?
6. **Yes**, do not ask any more questions and look at the predicted class for that node.
7. **No**, ask more questions and repeat.

Node attributes:
- `samples`: Counts how many training instances it applies to.
- `value`: Tells you how many training instances of each class this node applies to.
- `gini`: Measures its impurity.

> Note: Decision Trees do not require feature scaling or centering at all.

> Note: Scikit-Learn uses CART algorithm - only **binary trees** -> nonleaf nodes always have two children "yes/no answers".

> #### Model Interpretation: White Box Versus Black Box
>> - **White box models** are intuitive and easy to interpret (eg. Decision Trees).
>>
>> - **Black box models** are usually hard to explain in simple terms why the predictions were made (eg. Random Forests, neural networks).

## 6.3 Estimating Class Probabilities

A Decision Tree estimates the probability that an instance belongs to a particular class $k$ by:

1. Traverses the tree to find the leaf node for this instance
2. Returns the ratio of training instances of class $k$ in this node

For examples, a flower with (length=5cm, width=1.5cm) corresponds to depth-2 left leaf node. It should output the following probabilities:

- $0\%$ $(0/54)$ for *Iris setosa*
- $90.7\%$ $(49/54)$ for *Iris versicolor*
- $9.3\%$ $(5/54)$ for *Iris virginica*

In [12]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [13]:
tree_clf.predict([[5, 1.5]])

array([1])

## 6.4 The CART Training Algorithm

Scikit-Learn uses the **Classification and Regression Tree (CART)** to train Decision Trees (also called "growing trees").

It first splits the training set into two subsets using a single feature $k$ and a threshold $t_k$ (eg. "petal length <= 2.45 cm").

It chooses a pair $(k, t_k)$ such that it produces the purest subsets (minimizes the CART cost function).

And once it splits, it splits the subsets using the same logic, stopping once it reaches the maximum depth (`max_depth` hyperparameter) or cannot find a split that will reduce impurity.

These hyperparameters are additional stopping conditions:

- `min_samples_split`
- `min_samples_leaf`
- `min_weight_fraction_leaf`
- `max_leaf_nodes`

> Note: CART algorithm is a **greedy algorithm**. It greedily searches for an optimum split at the top level, then repeats at each subsequent level. It does not check whether or not the split will lead to the lowest possible impurity several levels down.

> Note: Greedy algorithms produce a solution that's reasonably good but not guaranteed to be optimal.

## 6.5 Computational Complexity

Since each node only requires checking the value of one feature, the overall prediction complexity is $O(log_2(m))$, independent of the number of features.  
=> Predictions are fast, even when dealing with large training sets.

Comparing all features on all samples at each node results in a training complexity of $ O(n \times m log_2 (m)) $.

## 6.6 Gini Impurity or Entropy?

Gini impurity measure is used by default, but **entropy impurity measure** can be used by setting `criterion="entropy"`.

In Machine Learning, entropy is frequently used as an impurity measure: a set's entropy is zero when it contains instances of only one class.

Most of the time it does not make a difference: they lead to similar trees. Gini impurity is slightly faster (good as default).

But Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

## 6.7 Regularization Hyperparameters

## 6.8 Regression

## 6.9 Instability